![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx.ai Text Extraction service to extract text from file

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code demonstrating how to run a Text Extraction job using python SDK and then retrieve the results in the form of JSON file.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The purpose of this notebook is to demonstrate the usage a Text Extraction service and `ibm-watsonx-ai` Python SDK to retrieve a text from file that is located at IBM Cloud Object Storage.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [COS connection](#cos_connection)
- [Text Extraction request](#text_extraction)
- [Results examination](#results)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install required packages

In [ ]:
!pip install "ibm-watsonx-ai>=1.1.5" | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [1]:
import getpass

from ibm_watsonx_ai import Credentials

credentials = Credentials(url="https://us-south.ml.cloud.ibm.com",
                          api_key=getpass.getpass("Please enter your WML api key (hit enter): "))

### Defining the project id
The Text Extraction service requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

### API Client initialization

In [3]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials=credentials, project_id=project_id)

<a id="cos_connection"></a>
## Create data connections with source document and results reference

The document, from which we are going to extract text, is located at IBM Cloud Object Storage (COS). In the following example we are going to use [Granite Code Models paper](https://arxiv.org/pdf/2405.04324) as a source text document. Also, the final results file, which will contain extracted text and necessary metadata, will be placed in COS. Therefore, we use `ibm_watsonx_ai.helpers.DataConnection` and `ibm_watsonx_ai.helpers.S3Location` class to create a Python objects that will represent the references to the processed files. Please note that you have to create connection asset with your COS details (for detailed explanation how to do this see [IBM Cloud Object Storage connection](https://dataplatform.cloud.ibm.com/docs/content/wsj/manage-data/conn-cos.html?context=wx) or check below cells).

### Create connection to COS
You can skip this section if you already have connection asset with **IBM Cloud Object Storage**.

In [4]:
datasource_name = 'bluemixcloudobjectstorage'
bucketname = "textextractionms"

In [5]:
cos_credentials = {
                  "endpoint_url": "<endpoint url>",
                  "apikey": "<apikey>",
                  "access_key_id": "<access_key_id>",
                  "secret_access_key": "<secret_access_key>"
              }

In [7]:
conn_meta_props= {
    client.connections.ConfigurationMetaNames.NAME: f"Connection to Database - {datasource_name} ",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: client.connections.get_datasource_type_id_by_name(datasource_name),
    client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to external Database",
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': bucketname,
        'access_key': cos_credentials['access_key_id'],
        'secret_key': cos_credentials['secret_access_key'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',
        'url': cos_credentials['endpoint_url']
    }
}

conn_details = client.connections.create(meta_props=conn_meta_props)
connection_asset_id = client.connections.get_id(conn_details)

Creating connections...
SUCCESS


### Upload file and create document and results reference

In [8]:
from ibm_watsonx_ai.helpers import DataConnection, S3Location

local_source_file_name = "granite_code_models_paper.pdf"
source_file_name = "./files/granite_code_models_paper.pdf"
results_file_name = "./files/text_extraction_granite_code_models_paper.json"

In [9]:
remote_document_reference = DataConnection(connection_asset_id=connection_asset_id,
                                           location=S3Location(bucket = bucketname, path = "."))
remote_document_reference.set_client(client)

remote_document_reference.write(local_source_file_name, remote_name=source_file_name)

Finally, we can create Data Connection that represents document and results reference.

In [10]:
document_reference = DataConnection(connection_asset_id=connection_asset_id,
                                    location=S3Location(bucket=bucketname,
                                                        path=source_file_name))

results_reference = DataConnection(connection_asset_id=connection_asset_id,
                                   location=S3Location(bucket=bucketname,
                                                       path=results_file_name))


<a id="text_extraction"></a>
## Text Extraction request

Since data connection for source and results files are ready, we can proceed to the text extraction run job step. To initialize Text Extraction manager we use `TextExtractions` class.

In [11]:
from ibm_watsonx_ai.foundation_models.extractions import TextExtractions
from ibm_watsonx_ai.metanames import TextExtractionsMetaNames

In [12]:
extraction = TextExtractions(api_client=client,
                             project_id=project_id)

When running job the steps for the text extraction pipeline can be specified. For more details about available steps see [documentation](https://cloud.ibm.com/apidocs/watsonx-ai#text-extraction). The list of steps available in sdk can be found below.

In [13]:
TextExtractionsMetaNames().show()

----------------  ----  --------
META_PROP NAME    TYPE  REQUIRED
OCR               dict  N
TABLE_PROCESSING  dict  N
----------------  ----  --------


To view sample parameter values for the text extraction steps run `get_example_values()`.

In [14]:
TextExtractionsMetaNames().get_example_values()

{'ocr': {'process_images': True, 'language_list': ['en']},
 'tables_processing': {'enabled': True}}

In our example we are going to use the following steps

In [15]:
steps = {TextExtractionsMetaNames.OCR: {'language_list': ['en']},
        TextExtractionsMetaNames.TABLE_PROCESSING: {'enabled': True}}

Now, we can run Text Extraction job.

In [16]:
details = extraction.run_job(document_reference=document_reference, 
                             results_reference=results_reference, 
                             steps=steps)
details

{'metadata': {'id': 'c23ee637-37c0-4d48-87a2-ebe6bdf0f38a',
  'created_at': '2024-08-01T08:38:01.738Z',
  'project_id': '18665143-ab93-434e-8805-2061fd58df40'},
 'entity': {'document_reference': {'type': 'connection_asset',
   'connection': {'id': '1ff26f06-a47c-4e9a-966d-c5c1409cc9a9'},
   'location': {'file_name': './files/granite_code_models_paper.pdf',
    'bucket': 'textextractionms'}},
  'results_reference': {'type': 'connection_asset',
   'connection': {'id': '1ff26f06-a47c-4e9a-966d-c5c1409cc9a9'},
   'location': {'bucket': 'textextractionms',
    'file_name': './files/text_extraction_granite_code_models_paper.json'}},
  'steps': {'ocr': {'languages_list': ['en']},
   'tables_processing': {'enabled': True}},
  'results': {'status': 'submitted',
   'running_at': '2024-08-01T08:38:01.738Z',
   'number_pages_processed': 0}}}

In [17]:
extraction_job_id = extraction.get_id(extraction_details=details)

We can list text extraction jobs using a proper list method.

In [19]:
extraction.list_jobs()

,EXTRACTION_ID,CREATED,STATUS
0,c23ee637-37c0-4d48-87a2-ebe6bdf0f38a,2024-08-01T08:38:01.738Z,completed


Moreover, to get details of a particular text extraction request run following

In [20]:
extraction.get_job_details(extraction_id=extraction_job_id)

{'entity': {'document_reference': {'connection': {'id': '1ff26f06-a47c-4e9a-966d-c5c1409cc9a9'},
   'location': {'bucket': 'textextractionms',
    'file_name': './files/granite_code_models_paper.pdf'},
   'type': 'connection_asset'},
  'results': {'completed_at': '2024-08-01T08:39:25.94612684Z',
   'number_pages_processed': 28,
   'running_at': '2024-08-01T08:38:01.738Z',
   'status': 'completed'},
  'results_reference': {'connection': {'id': '1ff26f06-a47c-4e9a-966d-c5c1409cc9a9'},
   'location': {'bucket': 'textextractionms',
    'file_name': './files/text_extraction_granite_code_models_paper.json'},
   'type': 'connection_asset'},
  'steps': {'ocr': {'languages_list': ['en']},
   'tables_processing': {'enabled': True}}},
 'metadata': {'created_at': '2024-08-01T08:38:01.738Z',
  'id': 'c23ee637-37c0-4d48-87a2-ebe6bdf0f38a',
  'project_id': '18665143-ab93-434e-8805-2061fd58df40'}}

Furthermore, to delete text extraction jub run use `delete_job()` method.

<a id="results"></a>
## Results examination

Once the job extraction is completed we can download the results file and process it further.

In [21]:
results_reference = extraction.get_results_reference(extraction_id=extraction_job_id)

In [22]:
filename = "text_extraction_results_granite_code_models_paper.json"

results_reference.download(filename=filename)

In [23]:
import json

metadata = json.load(open(filename, 'r'))
metadata.keys()

dict_keys(['metadata', 'styles', 'top_level_structures', 'all_structures'])

In [24]:
metadata.get('metadata')

{'num_pages': 28,
 'title': '',
 'keywords': '',
 'author': '',
 'publication_date': '',
 'subject': '',
 'charset': 'UTF-8'}

The following tokens were extracted from the source file. Note that we decided to print only 10 of them to keep the output clear and concise.

In [25]:
metadata.get('all_structures').get('tokens')[:10]

[{'id': 'TOKEN_8f9c63',
  'parent_id': 'PARA_b29579',
  'style_id': 'URWPalladioL_Roma_Black_9_0',
  'text': 'IBM',
  'bbox': {'page_number': 1,
   'x': 108.0,
   'y': 29.261946,
   'width': 18.869171,
   'height': 6.1070733}},
 {'id': 'TOKEN_8f124a',
  'parent_id': 'PARA_b29579',
  'style_id': 'URWPalladioL_Roma_Black_9_0',
  'text': 'Granite',
  'bbox': {'page_number': 1,
   'x': 409.57788,
   'y': 29.261946,
   'width': 33.23523,
   'height': 6.1070733}},
 {'id': 'TOKEN_c6c33e',
  'parent_id': 'PARA_b29579',
  'style_id': 'URWPalladioL_Roma_Black_9_0',
  'text': 'Code',
  'bbox': {'page_number': 1,
   'x': 445.30377,
   'y': 29.261946,
   'width': 23.362305,
   'height': 6.1070733}},
 {'id': 'TOKEN_7e8f6b',
  'parent_id': 'PARA_b29579',
  'style_id': 'URWPalladioL_Roma_Black_9_0',
  'text': 'Models',
  'bbox': {'page_number': 1,
   'x': 471.15674,
   'y': 29.261946,
   'width': 32.84671,
   'height': 6.1070733}},
 {'id': 'TOKEN_adaa70',
  'parent_id': 'SECTION_TITLE_a2d2a9',
  'styl

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!
 
 You learned how to use `TextExtractions` manager to run text extraction requests, check status of the submitted job and download a results file.
 
Check out our _<a href="https://ibm.github.io/watson-machine-learning-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors:
 **Mateusz Świtała**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.